In [ ]:
import google.generativeai as genai
import pandas as pd
import json
import re

# Configure the Generative AI model
genai.configure(api_key="Enter-API-KEY")
model = genai.GenerativeModel('gemini-1.5-flash')

# Function to generate test cases from a given user story and acceptance criteria in JSON format
def generate_test_cases(user_story, acceptance_criteria):
    prompt = f"""
        Generate test cases based on the user story and acceptance criteria in JSON format.
        User story: "{user_story}"
        Acceptance criteria: "{acceptance_criteria}"
        
        Format:
        {{
            "test_cases": [
                {{
                    "test_scenario": "Scenario description",
                    "test_case": "Test case description",
                    "steps": ["Step 1", "Step 2", "Step 3"],
                    "expected_result": "Expected outcome"
                }}
            ]
        }}
        
        Ensure the output is strictly in valid JSON format.
    """

    try:
        # Generate response from the model
        response = model.generate_content(contents=[prompt])
        response_text = response.text.strip()
        print(f"Generated Response (Raw):\n{response_text}\n")  # Debugging: Print raw response

        # Preprocess the response to remove unwanted characters
        response_text = re.sub(r'^```json|```$', '', response_text, flags=re.IGNORECASE).strip()
        print(f"Generated Response (Cleaned):\n{response_text}\n")  # Debugging: Print cleaned response

        # Parse the JSON response
        generated_data = json.loads(response_text)
        
        # Check if "test_cases" key exists
        if "test_cases" in generated_data:
            return generated_data["test_cases"]
        else:
            print(f"⚠️ No 'test_cases' key in response.")
            return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON Parsing Error: {e}\nResponse:\n{response_text}")
        return []
    except Exception as e:
        print(f"❌ Error generating test cases: {e}")
        return []

# Function to read user stories and acceptance criteria from an Excel file
def read_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        return df
    except Exception as e:
        print(f"❌ Error reading Excel file: {e}")
        return None

# Function to write data back to Excel
def write_excel(df, file_path):
    try:
        print(f"📝 Writing data to Excel file: {file_path}")
        print(f"📊 DataFrame to be written:\n{df}")
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
            df.to_excel(writer, index=False)
            # ✅ No need to manually close — `with` statement handles it
        print(f"✅ Data successfully written to {file_path}")
    except Exception as e:
        print(f"❌ Error writing to Excel: {e}")

# Function to add additional test cases as new rows in the dataframe
def add_additional_rows(df, index, test_cases):
    all_rows = df.to_dict('records')

    for test_case in test_cases[1:]:
        new_row = df.iloc[index].copy()
        new_row['Test Scenario'] = test_case.get("test_scenario", "")
        new_row['Test Case'] = test_case.get("test_case", "")
        new_row['Steps'] = "\n".join(test_case.get("steps", []))
        new_row['Expected Result'] = test_case.get("expected_result", "")
        all_rows.insert(index + 1, new_row)

    return pd.DataFrame(all_rows)

# Function to process the Excel and update with generated test cases
def process_excel(file_path):
    df = read_excel(file_path)
    if df is None:
        print("❌ Failed to read Excel file.")
        return
    
    print(f"📊 Original DataFrame shape: {df.shape}")

    # ✅ Ensure required columns exist
    for col in ['Test Scenario', 'Test Case', 'Steps', 'Expected Result']:
        if col not in df.columns:
            df[col] = ""  # Add missing columns as empty strings

    # ✅ Convert columns to string type to avoid dtype conflicts
    df['Test Scenario'] = df['Test Scenario'].astype(str)
    df['Test Case'] = df['Test Case'].astype(str)
    df['Steps'] = df['Steps'].astype(str)
    df['Expected Result'] = df['Expected Result'].astype(str)

    # ✅ Handle NaN values to avoid parsing issues
    df['User Story'] = df['User Story'].fillna('')
    df['Acceptance Criteria'] = df['Acceptance Criteria'].fillna('')

    # ✅ Create a copy of the original DataFrame
    modified_df = df.copy()
    row_offset = 0

    for index, row in df.iterrows():
        user_story = row['User Story']
        acceptance_criteria = row['Acceptance Criteria']

        if not user_story.strip():
            print(f"⚠️ Skipping row {index} due to empty user story.")
            continue

        print(f"🔎 Processing User Story: {user_story}\n")

        # ✅ Generate test cases using AI
        test_cases = generate_test_cases(user_story, acceptance_criteria)

        if not test_cases:
            print(f"⚠️ No test cases generated for User Story: {user_story}")
            continue

        # ✅ Update first row with the first test case
        first_test_case = test_cases[0]
        modified_df.at[index + row_offset, 'Test Scenario'] = first_test_case.get("test_scenario", "")
        modified_df.at[index + row_offset, 'Test Case'] = first_test_case.get("test_case", "")
        modified_df.at[index + row_offset, 'Steps'] = "\n".join(first_test_case.get("steps", []))
        modified_df.at[index + row_offset, 'Expected Result'] = first_test_case.get("expected_result", "")

        # ✅ Add additional test cases as new rows
        if len(test_cases) > 1:
            modified_df = add_additional_rows(modified_df, index + row_offset, test_cases)
            row_offset += len(test_cases) - 1

    print(f"📊 Final DataFrame shape after processing: {modified_df.shape}")

    # ✅ Write updated data back to Excel
    write_excel(modified_df, file_path)

# ✅ Example usage
file_path = r'Enter-Excel-Path'
process_excel(file_path)